# Load Model and Data

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch 
from dreamerv4.datasets import ShardedHDF5Dataset
from dreamerv4.models.utils import load_tokenizer
from dreamerv4.models.utils import load_denoiser
# DATA_PATH = "/home/mim-server/datasets/soar_data_sharded"
DATA_PATH = "/home/mim-server/datasets/pushcube-play-hdf5"
# DATA_PATH = "/home/mim-server/datasets/pushT/sharded"
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
resolution = (256, 256)

In [3]:
from hydra import initialize, compose
from omegaconf import OmegaConf
with initialize(version_base=None, config_path="../scripts/config"):
    cfg = compose(config_name="dynamics/finger-small.yaml")

/home/mim-server/miniconda3/envs/dreamerv4/lib/python3.11/site-packages/hydra/_internal/defaults_list.py:251: UserWarning: In 'dynamics/finger-small.yaml': Defaults list is missing `_self_`. See https://hydra.cc/docs/1.2/upgrades/1.0_to_1.1/default_composition_order for more information
  warnings.warn(msg, UserWarning)


In [4]:
dynamics_ckpt = "/home/mim-server/projects/rooholla/dreamer-v4-draft/checkpoints/dynamics_ckpts/finger-small.pt"
cfg.dynamics_ckpt = dynamics_ckpt

In [5]:
tokenizer_ckpt="/home/mim-server/projects/rooholla/dreamer-v4-draft/checkpoints/tokenizer_ckpts/finger.pt"
cfg.tokenizer_ckpt=tokenizer_ckpt

In [6]:
denoiser = load_denoiser(cfg, device, max_num_forward_steps=300)
tokenizer = load_tokenizer(cfg, device, max_num_forward_steps=300)
tokenizer = tokenizer.eval().cuda()
denoiser = denoiser.eval().cuda()

In [8]:
import mediapy
from torch.nn.functional import interpolate

dataset = ShardedHDF5Dataset(
        data_dir=DATA_PATH,
        window_size=16,
        stride=1,
        split='train',
        train_fraction=0.9,
        split_seed=123,
    )

# Arbitrary index into that episode
batch = dataset[6]
imgs = batch["image"]  # (T, C, H, W)
actions = batch["action"][:,:cfg.denoiser.n_actions]  # (1, T, N_lat, D_lat)
# actions=torch.zeros_like(actions)
imgs = interpolate(imgs, resolution).to(device=device)[None] # resize to tokenizer resolution

with torch.no_grad():
    with torch.autocast(device_type='cuda', dtype=torch.bfloat16):
        latents = tokenizer.encode(imgs)
        imgs_recon = tokenizer.decode(latents)

Train split: 49847 windows from 18 episodes


In [9]:
from mediapy import show_video
def plotVideo(video):
    imgs = video.cpu().permute(0,2,3,1).to(torch.float32).numpy()*255
    imgs = imgs.astype('uint8')
    mediapy.show_video(imgs, fps=10)

plotVideo(imgs_recon[0])
plotVideo(imgs[0])

This browser does not support the video tag.


This browser does not support the video tag.


In [10]:
from dreamerv4.sampling import forward_dynamics_no_cache, AutoRegressiveForwardDynamics

In [11]:
num_pred_steps=8
latents_ctx = latents[:, :4, ...]
cond_actions = actions[:latents_ctx.shape[1]+num_pred_steps][None]
cond_actions[:]=0
with torch.autocast(device_type='cuda', dtype=torch.bfloat16):
    z = forward_dynamics_no_cache(denoiser, 
                                latents_ctx,
                                cond_actions,
                                num_pred_steps=num_pred_steps,
                                num_diffusion_steps=128)

    with torch.no_grad():
        img_pred = tokenizer.decode(z)


plotVideo(img_pred[0].to(torch.float32))

In [11]:
world = AutoRegressiveForwardDynamics(denoiser, tokenizer, device=device, dtype=torch.bfloat16)
with torch.autocast(device_type="cuda", dtype=torch.bfloat16):
    world.reset(imgs[:,:8], actions[:8][None])

In [12]:
latents = []
imgs_pred = []
for i in range(32):
    action_t = actions[world.current_frame_index][None]
    # action_t[:, 0] = 1
    with torch.autocast(device_type="cuda", dtype=torch.bfloat16):
        img = world.step(action_t)
    latents.append(world.current_z.clone())
    imgs_pred.append(img)
z = torch.cat(latents, dim=1)
imgs_pred = torch.stack(imgs_pred, dim=1)

In [13]:
with torch.no_grad(), torch.autocast(device_type="cuda", dtype=torch.bfloat16):
    # z_decoded = tokenizer.decoder(world.latents_cond)
    z_decoded = tokenizer.decoder(z)
    imgs_recon = tokenizer.image_head(z_decoded)
    imgs_recon = (imgs_recon + 1.0) / 2.0  # [-1,1] → [0,1]
    imgs_recon = torch.clamp(imgs_recon, 0.0, 1.0)
plotVideo(imgs_recon[0])

This browser does not support the video tag.


In [14]:
imgs_pred.shape

torch.Size([1, 32, 3, 256, 256])

In [15]:
plotVideo(imgs_pred[0])

This browser does not support the video tag.


In [45]:
import cv2 
for i in range(imgs_pred):
    img = imgs_pred[0, 1]

TypeError: only integer tensors of a single element can be converted to an index

In [12]:
def ramp_weight(tau: torch.Tensor) -> torch.Tensor:
    """
    Eq. (8): w(τ) = 0.9 τ + 0.1

    tau: (B, T) or (B, T, 1, 1)
    returns: same shape as tau
    """
    return 0.9 * tau + 0.1

def compute_bootstrap_diffusion_loss_v1(
    info: dict,
    denoiser,
    continuous_actions,
):
    """
    info: dict from ForwardDiffusionWithShortcut.forward, with keys:
      x                   : (B, T, N_lat, D_latent)
      x_tau               : (B, T, N_lat, D_latent)
      tau                 : (B, T) float
      step                : (B, T) float
      tau_index           : (B, T) long
      step_index          : (B, T) long  (0 ↔ d_min, >0 ↔ larger steps)
      half_step_index     : (B, T) long
      tau_plus_half_index : (B, T) long
      d_min               : scalar float

    Returns:
      flow_loss      : scalar (flow branch, d = d_min)
      bootstrap_loss : scalar (bootstrap branch, d > d_min)
    """
    x = info["x"]                         # (B, T, N_lat, D_lat)
    x_tau = info["x_tau"]                 # (B, T, N_lat, D_lat)
    tau = info["tau"]                     # (B, T) float
    step = info["step"]                   # (B, T) float
    tau_index = info["tau_index"]         # (B, T) long
    step_index = info["step_index"]       # (B, T) long (0..max_pow2)
    half_step_index = info["half_step_index"]              # (B, T) long
    tau_plus_half_index = info["tau_plus_half_index"]      # (B, T) long
    d_min = info["d_min"]                 # scalar float

    B, T, N_lat, D_lat = x.shape

    tau_b = tau.view(B, T, 1, 1)          # (B, T, 1, 1)
    step_b = step.view(B, T, 1, 1)        # (B, T, 1, 1)

    # -------------------------------------------------
    # 1) Big-step prediction:  \hat z_1 = f_θ(z_τ, τ, d)
    # -------------------------------------------------

    z_hat = denoiser(
            noisy_z=x_tau,
            action=continuous_actions,
            sigma_idx=tau_index,
            step_idx=step_index,
        )    # -> (B, T, N_lat, D_lat)

    # =================================================
    # 2) Flow loss branch (d = d_min): || ẑ_1 - z_1 ||^2
    # =================================================
    flow_residual = z_hat - x                      # (B, T, N_lat, D_lat)
    # average over tokens & dims: (B, T)
    flow_sq = flow_residual.pow(2).mean(dim=(-1, -2))

    # d = d_min ↔ step_index == 0 in this convention
    mask_small = (step_index == 0).float()         # (B, T), 1 where d = d_min

    # ramp weight w(τ) from Eq. (8), τ ∈ [0, 1]
    w_tau = ramp_weight(tau)                       # (B, T)

    flow_loss_per = w_tau * flow_sq * mask_small  # (B, T)
    denom_flow = mask_small.sum().clamp_min(1.0)  # avoid div by 0
    flow_loss = flow_loss_per.sum() / denom_flow  # scalar

    # =================================================
    # 3) Bootstrap branch (d > d_min):
    #    v_target = (b1 + b2)/2, with
    #    b1 = [f(z_τ, τ, d/2) - z_τ] / (1 - τ)
    #    b2 = [f(z', τ + d/2, d/2) - z'] / (1 - (τ + d/2))
    #    Loss: (1 - τ)^2 || v_hat - sg(v_target) ||^2 * w(τ)
    # =================================================
    with torch.no_grad():
        # ------- First half-step -------
        f1 = denoiser(
            noisy_z=x_tau,
            action=continuous_actions,
            sigma_idx=tau_index,
            step_idx=half_step_index,
        )    # -> (B, T, N_lat, D_lat)

        # b1: (B, T, N_lat, D_lat)
        b1 = (f1 - x_tau) / (1.0 - tau_b)

        # z' = z_τ + b1 * d/2
        z_prime = x_tau + b1 * (step_b / 2.0)      # (B, T, N_lat, D_lat)

        # ------- Second half-step -------
        f2 = denoiser(
            noisy_z=z_prime,
            action=continuous_actions,
            sigma_idx=tau_plus_half_index,
            step_idx=half_step_index,
        )    # -> (B, T, N_lat, D_lat)

        denom2 = 1.0 - (tau_b + step_b / 2.0)      # (B, T, 1, 1)
        b2 = (f2 - z_prime) / denom2               # (B, T, N_lat, D_lat)

        # v_target: average of the two half-step velocities
        v_target = 0.5 * (b1 + b2)                 # (B, T, N_lat, D_lat)  (sg via no_grad)

    # predicted velocity from big step:
    v_hat = (z_hat - x_tau) / (1.0 - tau_b)        # (B, T, N_lat, D_lat)

    boot_err = v_hat - v_target                    # (B, T, N_lat, D_lat)

    # x-space scaling (1 - τ)^2 and ramp weight w(τ)
    boot_sq = ((1.0 - tau_b) ** 2 * boot_err.pow(2)).mean(dim=(-1, -2))  # (B, T)

    mask_large = (step_index > 0).float()          # (B, T)  d > d_min
    boot_loss_per = w_tau * boot_sq * mask_large   # (B, T)

    denom_boot = mask_large.sum().clamp_min(1.0)
    bootstrap_loss = boot_loss_per.sum() / denom_boot

    return flow_loss, bootstrap_loss

In [13]:
from dreamerv4.loss import ForwardDiffusionWithShortcut

diffuser = ForwardDiffusionWithShortcut(128)

In [14]:
info = diffuser(latents.cuda())

In [15]:
with torch.no_grad():
    l1,  l2 = compute_bootstrap_diffusion_loss_v1(info, denoiser, actions.cuda())
    print(l1, l2)

tensor(0.0078, device='cuda:0') tensor(0.0002, device='cuda:0')


In [16]:
from dreamerv4.loss import compute_bootstrap_diffusion_loss

l1, l2 = compute_bootstrap_diffusion_loss(info, denoiser, actions.cuda())
print(l1, l2)
l = l1+l2
l.backward()
g1= denoiser.model.layers[0].layers[0].attn.attn.W_q.weight.grad.clone()
denoiser.zero_grad()


tensor(0.0078, device='cuda:0', grad_fn=<DivBackward0>) tensor(0.0002, device='cuda:0', grad_fn=<DivBackward0>)


In [18]:
l1, l2 = compute_bootstrap_diffusion_loss_v1(info, denoiser, actions.cuda())
print(l1, l2)
l = l1+l2
l.backward()
g2= denoiser.model.layers[0].layers[0].attn.attn.W_q.weight.grad.clone()
denoiser.zero_grad()


tensor(0.0078, device='cuda:0', grad_fn=<DivBackward0>) tensor(0.0002, device='cuda:0', grad_fn=<DivBackward0>)


In [22]:
(g1-g2).norm()

tensor(1.4568e-09, device='cuda:0')